In [1]:
import os
import shutil
import cv2
import numpy as np
from keras.models import load_model

# Step 1: Load the pre-trained model
model_path = "masknet.keras"  # Adjust this path to your model file
model = load_model(model_path)

# Function to check if a file is a valid image
def is_image_file(file):
    valid_extensions = [".jpg", ".jpeg", ".png", ".bmp"]
    return any(file.lower().endswith(ext) for ext in valid_extensions)

# Step 2: Define helper functions for loading and processing images
def load_and_preprocess_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    
    # Check if the image was loaded correctly
    if img is None:
        print(f"Warning: Could not load image {image_path}. Skipping file.")
        return None
    
    # Resize the image to 128x128 as per your model's input requirements
    img = cv2.resize(img, (128, 128))
    img = img.astype('float32')
    img /= 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Step 3: Iterate over the directory and check for masked faces
def check_masked_images(source_dir, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for subdir, dirs, files in os.walk(source_dir):
        for file in files:
            if not is_image_file(file):
                print(f"Skipping non-image file: {file}")
                continue
            
            file_path = os.path.join(subdir, file)
            
            # Process image
            img = load_and_preprocess_image(file_path)
            
            # If the image couldn't be loaded, skip it
            if img is None:
                continue
            
            # Make prediction (assuming model output is binary: 0 = no mask, 1 = masked)
            prediction = model.predict(img)
            is_masked = prediction[0][0] > 0.5  # Adjust threshold as needed

            if is_masked:
                # Create corresponding directory structure in destination folder
                relative_path = os.path.relpath(subdir, source_dir)
                dest_subdir = os.path.join(dest_dir, relative_path)
                if not os.path.exists(dest_subdir):
                    os.makedirs(dest_subdir)
                # Copy image to the new directory
                shutil.copy(file_path, os.path.join(dest_subdir, file))

# Step 4: Use the function
source_directory = "test"  # Directory containing the photos
destination_directory = "test_result"  # Directory to save masked photos

check_masked_images(source_directory, destination_directory)
